In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pickle
import time
import requests
import os
from scraping import *
import pandas as pd
import regex as re
import numpy as np
import random

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:

chromedriver = "E:/chromedriver_win32/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)


In [151]:
url = "https://www.tripadvisor.in/Attraction_Review-g61000-d139187-Reviews-or5-Glacier_Point-Yosemite_National_Park_California.html#REVIEWS"
driver.get(url)
time.sleep(10)
button = driver.find_element_by_class_name("_3aUwQbpX")
button.click()
ta_reviews_page_soup = BeautifulSoup(driver.page_source,'html5lib')


In [4]:
url = "https://www.tripadvisor.in/Attraction_Review-g61000-d139187-Reviews-or{}-Glacier_Point-Yosemite_National_Park_California.html#REVIEWS"

col_names = ['attraction_name', 
 'attraction_id',
 'user_name',
 'user_profile_link',
 'review_date',
 'helpful_votes',
 'rating',
 'review_link',
 'review_text',
 'review_title',
 'experience_date']

df_reviews = pd.DataFrame(columns=col_names)

# point_name = soup.find('title').text.split(' - ')[0]
# point_id = '-'.join(url.split('-')[1:3])

In [29]:
df_reviews['review_text'][500]

&quot;I&#39;d seen the pictures before we visited from England and assumed they must have been touched up. But this is breathtaking. Came twice as the first time it was a bit hazy due to forest fires.  Sunset here is hard to describe. If you ever get the chance go!&quot;

In [11]:
attraction_url_list = ['https://www.tripadvisor.in/Attraction_Review-g61000-d489919-Reviews-or{}-Yosemite_Valley-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d592681-Reviews-or{}-Mariposa_Grove_of_Giant_Sequoias-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d139183-or{}-Reviews-Half_Dome-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d593212-or{}-Reviews-Tunnel_View-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d146103-or{}-Reviews-Tioga_Pass-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d139182-or{}-Reviews-El_Capitan-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d483482-or{}-Reviews-Mist_Trail-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d483476-or{}-Reviews-Yosemite_Falls-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d483481-or{}-Reviews-Vernal_Fall-Yosemite_National_Park_California.html']

attraction_name_list = [re.sub('_', ' ', name.split('-')[-2]) for name in attraction_url_list]

attraction_numbers = [2000, 1861, 1472, 2000, 1159, 925, 875, 1760, 1145]

attraction_id_list = ['-'.join(url.split('-')[1:3]) for url in attraction_url_list]

In [30]:
df_reviews_total = pd.read_csv('attraction_point_reviews.csv')
# df_reviews_total.tail()

## Come back & get El Capitan reviews from 425 to 925!!

In [43]:

for i in range(6,9):

    url = attraction_url_list[i]
    point_name = attraction_name_list[i]
    point_id = attraction_id_list[i]
    n = attraction_numbers[i]
    df_reviews = pd.DataFrame(columns=col_names) 

    for j in range(0, n, 5):

        try:
            response = requests.get(url.format(j))
            if response.status_code == 200:

                soup = BeautifulSoup(response.text, 'html5lib')
                reviews_list = ta_attraction_reviews_parser(soup)
                
                if reviews_list:
                    df_reviews = df_reviews.append(reviews_list, ignore_index = True)

                    df_reviews['attraction_name'] = point_name
                    df_reviews['attraction_id'] = point_id

                    df_reviews.to_csv(f"attraction_point_reviews_{point_name}.csv" , index = False)

                else:
                    print(f"Reviews List came up empty on url {url}")
                    # input()

            else:
                print(f"status code returned is {response.status_code} for page {url}")
                # input()
        
        except Exception as err_message:
            print(f"For page {url}, received following error message: {err_message}")

        # print(df_reviews.tail(2))
        # time.sleep(0.5*random.random())

    df_reviews.to_csv(f"attraction_point_reviews_{point_name}.csv" , index = False)
    df_reviews.tail()

    df_reviews_total = pd.concat([df_reviews_total, df_reviews])
    df_reviews_total.to_csv("attraction_point_reviews.csv", index = False)

df_reviews_total.tail()
             


    


 on url https://www.tripadvisor.in/Attraction_Review-g61000-d483482-or{}-Reviews-Mist_Trail-Yosemite_National_Park_California.html
Reviews List came up empty on url https://www.tripadvisor.in/Attraction_Review-g61000-d483482-or{}-Reviews-Mist_Trail-Yosemite_National_Park_California.html
Reviews List came up empty on url https://www.tripadvisor.in/Attraction_Review-g61000-d483482-or{}-Reviews-Mist_Trail-Yosemite_National_Park_California.html
Reviews List came up empty on url https://www.tripadvisor.in/Attraction_Review-g61000-d483482-or{}-Reviews-Mist_Trail-Yosemite_National_Park_California.html
Reviews List came up empty on url https://www.tripadvisor.in/Attraction_Review-g61000-d483482-or{}-Reviews-Mist_Trail-Yosemite_National_Park_California.html
Reviews List came up empty on url https://www.tripadvisor.in/Attraction_Review-g61000-d483482-or{}-Reviews-Mist_Trail-Yosemite_National_Park_California.html
Reviews List came up empty on url https://www.tripadvisor.in/Attraction_Review-g6100

,attraction_name,attraction_id,user_name,user_profile_link,review_date,helpful_votes,rating,review_link,review_text,review_title,experience_date
1139,Vernal Fall,g61000-d483481,Fairport Travelers,/Profile/Clarkvara,Jul 2008,560,4,/ShowUserReviews-g61000-d483481-r17854214-Vern...,Vernal Falls is sort of like the first leg of ...,Nice not too long hike,None
1140,Vernal Fall,g61000-d483481,CAtravelfamily,/Profile/CAtravelfamily,Jun 2008,16,5,/ShowUserReviews-g61000-d483481-r17268549-Vern...,"Whew, it was a tough climb at times, but once ...",Worth the effort!,None
1141,Vernal Fall,g61000-d483481,doodlebugakj,/Profile/doodlebugakj,Jul 2007,NaN,5,/ShowUserReviews-g61000-d483481-r8255121-Verna...,This was a really fun hike but when we came ar...,Wow that was a lot of stairs!,None
1142,Vernal Fall,g61000-d483481,Jase2153,/Profile/Jase2153,Sep 2005,9,5,/ShowUserReviews-g61000-d483481-r3910762-Verna...,I was visiting Yosemite from Australia and wen...,Worth The Trip,None
1143,Vernal Fall,g61000-d483481,booradley2,/Profile/booradley2,Sep 2004,214,5,/ShowUserReviews-g61000-d483481-r2512847-Verna...,I've never been especially enthusiastic about ...,Do Not Miss Vernal Fall,None


In [42]:
df_reviews_total

,attraction_name,attraction_id,user_name,user_profile_link,review_date,helpful_votes,rating,review_link,review_text,review_title,experience_date
0,Glacier Point (Yosemite National Park),g61000-d139187,Lisa,/Profile/lisa_1i,Oct 2020,2.0,5,/ShowUserReviews-g61000-d139187-r774573486-Gla...,"Not too many ppl when we arrive at 3pm, probab...",Glacier point during pandemic,October 2020
1,Glacier Point (Yosemite National Park),g61000-d139187,Cali4niahiker,/Profile/Cali4niahiker,Oct 2020,NaN,5,/ShowUserReviews-g61000-d139187-r774377599-Gla...,Glacier Point at about 7000 ft elevation can b...,Been here dozens of times. Never gets old!,October 2020
2,Glacier Point (Yosemite National Park),g61000-d139187,Gus6,/Profile/Gus6,Oct 2020,67.0,5,/ShowUserReviews-g61000-d139187-r774013232-Gla...,"The most Iconic view of the half dome, this is...","The most Iconic view of the half dome, this is...",November 2019
3,Glacier Point (Yosemite National Park),g61000-d139187,Cutecpl,/Profile/cutecpl,Sep 2020,2.0,5,/ShowUserReviews-g61000-d139187-r771781116-Gla...,What an absolute breathtaking view! Such a fu...,Glacier Point,November 2019
4,Glacier Point (Yosemite National Park),g61000-d139187,johnnnita,/Profile/johnnnita,Sep 2020,NaN,5,/ShowUserReviews-g61000-d139187-r770059892-Gla...,This attraction/view will truly take your brea...,THE # ONE PANORAMIC VIEW IN YOSEMITE NATIONAL ...,August 2020
...,...,...,...,...,...,...,...,...,...,...,...
420,El Capitan,g61000-d139182,Sam A,/Profile/114sama,Sep 2016,84.0,5,/ShowUserReviews-g61000-d139182-r418076310-El_...,Massive sheer rock formation that is amazing t...,Majestic,September 2016
421,El Capitan,g61000-d139182,Bernadette C,/Profile/550bernadettec,Sep 2016,80.0,5,/ShowUserReviews-g61000-d139182-r416184423-El_...,A beautiful granite structure. Photo worthy fo...,Majestic sight to behold,September 2016
422,El Capitan,g61000-d139182,rhymes0102,/Profile/rhymes0102,Sep 2016,65.0,5,/ShowUserReviews-g61000-d139182-r415916042-El_...,Don't miss El Capitan whilst at the valley. Yo...,A must!,September 2016
423,El Capitan,g61000-d139182,Roxana_DTS,/Profile/Roxana_DTS,Sep 2016,155.0,5,/ShowUserReviews-g61000-d139182-r415109515-El_...,I think this one of the nicest views in the pa...,Stunning views!,August 2016
